In [47]:
%cd ..
%rm -rf mt-dnn_port
!git clone https://github.com/jubs12/mt-dnn_port.git -b seed
%cd mt-dnn_port/

/content
Cloning into 'mt-dnn_port'...
remote: Enumerating objects: 8652, done.
remote: Counting objects: 100% (8652/8652), done.
remote: Compressing objects: 100% (7856/7856), done.
remote: Total 63042 (delta 1234), reused 8082 (delta 743), pack-reused 54390
Receiving objects: 100% (63042/63042), 493.87 MiB | 24.21 MiB/s, done.
Resolving deltas: 100% (26338/26338), done.
Checking out files: 100% (27943/27943), done.
/content/mt-dnn_port


In [0]:
modes = [
         'st-dnn/assin2-rte',
         'st-dnn/assin2-sts',
         'st-dnn/assin-1+2-rte', 
         'st-dnn/assin-1+2-sts',
         'st-dnn/assin-ptbr+2-sts', #tem outlier
         'st-dnn/assin-ptbr+2-rte',
         'st-dnn/assin2-rte',
         'st-dnn/assin-ptbr-sts',
         'st-dnn/assin-ptbr-rte',
         'st-dnn/assin-ptpt-sts',
         'st-dnn/assin-ptpt-rte',
         'st-dnn/tweetsent',
         'mt-dnn_assin', #falta
         'mt-dnn_assin-ptbr+assin2', #falta
         'mt-dnn_assin2', #falta
         'mt-dnn_assin+tweetsent', #falta
]

In [55]:
import statistics
import pprint
import json
import os

SEEDS = range(2016,2021)
SUFIX = '_test_scores_4.json'
THRESHOLD = 10

grad_norms = ['1.0', '2.0', '5.0']
dropouts = ['0.1', '0.3', '0.05']


global COUNT
COUNT = 0

global TOTAL
TOTAL = 0
def get_metrics(mode,pretrained, grad_norm, dropout):
    global COUNT
    global TOTAL
    metrics = dict()
    for seed in SEEDS:
        folder = f'output/{mode}/{pretrained}/seed/{seed}/grad_norm/{grad_norm}/dropout/{dropout}'
        filenames = [f for f in os.listdir(folder) if f.endswith(SUFIX)]
        
        for filename in filenames:
            filepath = f'{folder}/{filename}'
            
            with open(filepath) as f:
                info = json.load(f)
        
            task = filename.replace(SUFIX, '')
            main_metric = 'ACC' if 'ACC' in info['metrics'].keys() else 'Pearson'
            
            if not info['metrics']:
                return None
            else:
                TOTAL = TOTAL + 1


            task_metric = info['metrics'][main_metric]

            if task not in metrics.keys():
                metrics[task] = {'metrics': [task_metric]}
            else: 
                metrics[task]['metrics'].append(task_metric) 

    for task in metrics.keys():
        avg = statistics.mean(metrics[task]['metrics'])
        std = statistics.stdev(metrics[task]['metrics'])

        metrics[task].update({'avg': avg, 'std': std})

        outliers = [seed for (idx, seed) in enumerate(SEEDS) 
                    if abs(metrics[task]['metrics'][idx] - avg) > THRESHOLD]

        COUNT += len(outliers)

        metrics[task].update({'outliers': outliers})

    return metrics


result = dict()
for mode in modes:
    print(mode)
    result[mode] = dict()
    for pretrained in os.listdir(f'output/{mode}'):    
        result[mode][pretrained] = {grad_norm: 
                            {dropout: get_metrics(mode,pretrained, grad_norm, dropout) for dropout in dropouts}
                            for grad_norm in grad_norms}    

print(f'{COUNT}: {COUNT/TOTAL*100}%')

st-dnn/assin2-rte
st-dnn/assin2-sts
st-dnn/assin-1+2-rte
st-dnn/assin-1+2-sts
st-dnn/assin-ptbr+2-sts
st-dnn/assin-ptbr+2-rte
st-dnn/assin2-rte
st-dnn/assin-ptbr-sts
st-dnn/assin-ptbr-rte
st-dnn/assin-ptpt-sts
st-dnn/assin-ptpt-rte
st-dnn/tweetsent
mt-dnn_assin
mt-dnn_assin-ptbr+assin2
mt-dnn_assin2
mt-dnn_assin+tweetsent
9: 0.11389521640091116%
